In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('dataset/sms.tsv', header=None, delimiter='\t', names=['label','Messages'])

In [3]:
data.head(20)

,label,Messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [4]:
data.shape

(5572, 2)

In [5]:
data['label'] = data['label'].map({'ham':0,'spam':1})

In [6]:
data.head()

,label,Messages
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
X = data.Messages
y = data.label

In [8]:
X.isnull().sum()

0

In [9]:
y.isnull().sum()

0

In [30]:
from sklearn.datasets import load_digits
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold,cross_val_score
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

In [11]:
models=[LogisticRegression(), MultinomialNB()]
kf = KFold(n_splits=10,shuffle=True)

In [13]:
c = CountVectorizer(stop_words='english')
X_new = c.fit_transform(X)

In [14]:
for model in models:
    scores = cross_val_score(model, X_new, y,cv=kf,scoring='recall')
    print(np.mean(scores))
    

0.8626005686453448
0.9528021615717048


In [15]:
s = cross_val_score(MultinomialNB(), X_new,y, cv=KFold(10, shuffle=True), scoring='recall')

In [16]:
log = cross_val_score(LogisticRegression(), X_new, y, cv = KFold(10,shuffle=True), scoring='recall')

In [17]:
print(np.mean(s))
print(np.mean(log))

0.9538914104044448
0.8597008427137798


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
type(X_train)

pandas.core.series.Series

In [19]:
pipe1 = make_pipeline(CountVectorizer(), MultinomialNB())
pipe2 = make_pipeline(CountVectorizer(), LogisticRegression())

In [20]:
pipe1.fit(X_train, y_train)
pipe2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [21]:
pred1 = pipe1.predict(X_test)
pred2 = pipe2.predict(X_test)

In [22]:
score1 = recall_score(y_test, pred1)
score2 = recall_score(y_test, pred2)

In [23]:
print(score1)
print(score2)

0.9243243243243243
0.8702702702702703


In [24]:
cm = confusion_matrix(y_test, pred1)
cm

array([[1206,    2],
       [  14,  171]])

In [25]:
cm1 = confusion_matrix(y_test, pred2)
cm1

array([[1202,    6],
       [  24,  161]])

In [28]:
tfidf = TfidfVectorizer(stop_words='english')

In [34]:
param_grid = {'clf__C':[1.0,10.0,100.0]}

In [35]:
pip = Pipeline([('vect', tfidf)])
gs_lr_tfidf = GridSearchCV(pip, param_grid, scoring='accuracy')

In [44]:
X_tfidf = tfidf.fit_transform(X_train)

In [47]:
X_test_tfidf = tfidf.transform(X_test)

In [45]:
nb1 = MultinomialNB()
nb1.fit(X_tfidf, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [48]:
pred_tf = nb1.predict(X_test_tfidf)

In [50]:
nb1.score(X_test_tfidf, y_test)

0.9755922469490309